In [ ]:
import pymongo as pm
import sys
import csv 
import datetime as dt
import pandas as pd
import MySQLdb as db
import time
import numpy as np

In [ ]:
# importing bokeh-specific modules
from bokeh.plotting import figure
from bokeh.models import HoverTool, Range1d, DatetimeTickFormatter, NumeralTickFormatter,LinearAxis
from bokeh.charts import TimeSeries, output_file, output_notebook, save, show
from bokeh.resources import INLINE #to render plot offline

# output within notebook
output_notebook(resources=INLINE)  #to render plot offline
# output to html file
# output_file('test.html')

# importing plotly-specific modules
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=False) #load plotly.js from local package


In [ ]:
def plotly_plot(smartmeter_collection_list):
    
    final_trace = []

    for coll in smartmeter_collection_list:
        #print coll
        trace = go.Scatter(x=dict_timestamp[coll], y=dict_val[coll], name=coll)

        #print dict_timestamp[coll]
        final_trace.append(trace)

    # Edit the layout
    layout = dict(
              title = 'kresit smartmeter dataViz',
              xaxis = dict(title = 'timestamp'),
              yaxis = dict(title = 'Values'),
              )
    # Plot and embed in ipython notebook
    fig = dict(data=final_trace, layout=layout)

    plotly.offline.iplot(fig, show_link=False) #with layout 


def bokeh_plot(smartmeter_collection_list):
    colors_list = ['Blue','Brown','Red','Green','Cyan','Black','Silver','Red','RosyBrown','RoyalBlue','SaddleBrown','Salmon','SandyBrown','SeaGreen']

    p = configure_bokeh_plot_options() #create a new figure and configure it

    c=0
    for coll in smartmeter_collection_list:
        print coll
        p.line(dict_timestamp[coll],dict_val[coll], legend = coll, color=colors_list[c])
        c = (c + 1)%len(colors_list)
    show(p)


# configure plot options
def configure_bokeh_plot_options():
    
    # print len(colors_list)
    
    # interactive options that we need to shown besides the plot
    TOOLS = "pan, box_zoom, wheel_zoom, resize, reset, save, hover, undo, redo"  #this method works however does not allow setting properties of the tools
    
    # p = figure(title="seil temperature", x_axis_type="datetime", x_axis_label='time', y_axis_label='temperature', tools=TOOLS)

    # create a new figure with a title and axis labels
    p = figure(x_axis_label='Time', y_axis_label='Power', plot_width=850, plot_height=650, tools=TOOLS,x_axis_type="datetime", logo=None)
    # p.extra_y_ranges = {"foo": Range1d(start=15, end=30)}
    # p.add_layout(LinearAxis(y_range_name="foo"), 'right')
    # rotate x-ticks by 45 degrees
    p.xaxis.major_label_orientation=np.pi/4

    p.background_fill_color = "beige"
    p.background_fill_alpha = 0.5

    # show the tooltip
    # p.select(dict(type=HoverTool)).tooltips = {"x":"$x", "y":"$y"}
    hover = p.select(dict(type=HoverTool))
    # hover.tooltips = {"x":"$x", "y":"$y"} #works
    hover.tooltips = {"time":"@x", "y":"@y{0.00}"} #works better

    p.ygrid.minor_grid_line_color = 'navy'
    p.ygrid.minor_grid_line_alpha = 0.1
    
    return p

In [ ]:
smartmeter_collection_list = [
  'power_k_p',
#     'power_k_seil_a',
#     'power_k_seil_p',
#     'power_k_a',
#     'power_k_wc_p',
#     'power_k_wc_l',
#     'power_k_wc_a',
#     'power_k_lab_od1',
#     'power_k_lab_od2',
#     'power_k_lab_od3',
#     'power_k_clsrm_ac1',
#     'power_k_clsrm_ac2',
#     'power_k_clsrm_ac3'
]

# params = ["_id","FwdWh","PF1","PF3","PF2","FwdVARh","A3","V23","VLL","VLN","Timestamp","V3","A1","V1","V2","A2","PF","VA2","VA3","VA1","A","VA","FwdVAh","F","W","VAR","V31","VAR1","VAR3","VAR2","V12","W3","W2","W1"]
params = ['TS','W']

def connect_mongo():
    db_mo = pm.MongoClient("10.129.23.41",27017)
    con = db_mo['data']  #new database
    return db_mo,con

# for displaying params column in mongo query
def get_param_dict(params):
    tempd = {}
    for param in params:
        tempd[param] = 1

    tempd['_id']= False
    #print tempd

    return tempd



In [ ]:
if __name__ == '__main__':
    # enter year, month and day accordingly to get the data
    year = 2017
    month = 1 
    day = 27

    start_hr = 14; start_min = 0
    end_hr = 15; end_min = 0

    start_ts = int(dt.datetime(year, month, day, start_hr, start_min,0 ).strftime('%s'))
    if end_hr != -1:
        end_ts = int(dt.datetime(year, month, day, end_hr, end_min,0 ).strftime('%s'))
    else:
        end_ts = int(time.time())

    # print start and end time epochs
    print start_ts, end_ts
    mo_db, mo_con = connect_mongo() # connection to mongo db

    dict_val = {}
    dict_timestamp = {}

    param_dict = get_param_dict(params)
    # print mo_con.collection_names()

    for key in smartmeter_collection_list:
        print "Adding data from collection %s" %(key)
        df = pd.DataFrame(list(mo_con[key].find({"TS":{"$gt":start_ts,"$lt":end_ts}},param_dict)))

        dict_timestamp[key] = df['TS'].map(dt.datetime.fromtimestamp)
        dict_val[key] = df['W']
    
    mo_db.close()

In [ ]:
plotly_plot(smartmeter_collection_list)
# bokeh_plot(collection_list)